In [18]:
import torch 
import torch.nn as nn
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import sys
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [19]:
writer = SummaryWriter('runs/mnist')

In [28]:
LR = 0.001
EPOCH = 1
BATCH_SIZE = 64

In [21]:
train_data = MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [22]:
train_loader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=BATCH_SIZE,
)

test_loader = DataLoader(
    test_data,
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [23]:
class Net(nn.Module):
    def __init__(self, input, hidden, output):
        super(Net, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input, hidden),
            nn.ReLU(),
            nn.Linear(hidden, output)
        )
    
    def forward(self, x):
        x = self.layer(x)
        return x

In [25]:
net = Net(28*28, 500, 10)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(net.parameters(), lr=LR)

In [27]:
examples = iter(train_loader)
example_data, example_target = examples.next()

writer.add_graph(net, example_data.reshape(-1, 28*28))

In [30]:
running_loss = 0.0
running_correct = 0
total_steps = len(train_loader)
for i in range(EPOCH):
    for step, (x,y) in enumerate(train_loader):

        x = x.reshape(-1, 28*28)
        y_pred = net(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()

        running_loss += loss.item()
        _, predicted = torch.max(y_pred.data, 1)

        if step+1 % 100 == 0:
            print (f'Epoch [{i+1}/{EPOCH}], Step [{step+1}/{total_steps}], Loss: {loss.item():.4f}')
            writer.add_scalar('training loss', running_loss /100, i* total_steps + step)
            running_accuracy = running_correct / 100 / predicted.size(0)
            writer.add_scalar('accuracy', running_accuracy, i * total_steps + step)
            running_correct = 0
            running_loss = 0.0


In [31]:
class_labels = []
class_preds = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        labels = labels
        outputs = net(images)
        # max returns (value ,index)
        values, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        class_probs_batch = [F.softmax(output, dim=0) for output in outputs]

        class_preds.append(class_probs_batch)
        class_labels.append(predicted)

    # 10000, 10, and 10000, 1
    # stack concatenates tensors along a new dimension
    # cat concatenates tensors in the given dimension
    class_preds = torch.cat([torch.stack(batch) for batch in class_preds])
    class_labels = torch.cat(class_labels)

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 97.57 %


In [32]:
classes = range(10)
for i in classes:
    labels_i = class_labels == i
    preds_i = class_preds[:, i]
    writer.add_pr_curve(str(i), labels_i, preds_i, global_step=0)
    writer.close()